In [1]:
#import some useful packages

import numpy as np
from qiskit.visualization import array_to_latex
import matplotlib.pyplot as plt
from qiskit_ibm_provider import IBMProvider
from qiskit.circuit import Gate, Parameter, QuantumCircuit
from qiskit import pulse, schedule
from qiskit.tools.jupyter import *
from qiskit_ibm_provider.job import job_monitor
from qiskit.pulse import Gaussian
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
%config InlineBackend.figure_formats = ['svg']
import time
import os
import csv

import warnings
warnings.filterwarnings('ignore')

import time
import os
import csv

IBMProvider.save_account('c6f7fbd78215239dcf45c2777fa3cc29e68f12ad9e9ded1195e2837c35dd449cfce4ff51c3eb461e2bbcd8efb820f615600fcbdd8330a2093e74a3dbb370aaad', overwrite=True)
provider = IBMProvider()
backend = provider.get_backend('ibm_lagos')

In [2]:
#get information about backend
backend_defaults = backend.defaults()
backend_properties = backend.properties()
backend_config = backend.configuration()

In [3]:
#hyperparameters
qubit = 0
f01 = backend.properties().qubits[qubit][2].value * 1e9
anhar = backend.properties().qubits[qubit][3].value * 1e9
f12 = f01 + anhar

qubit = 0
pi = np.pi
cos = np.cos
sin = np.sin
exp = np.exp
NUM_SHOTS = 20000
scale_factor = 1e-7

amp12 = 0.28992251826581195
dur12 = 144
sig12 = dur12/4
dur01 = 160
sig01 = dur01/4

In [4]:
#some useful function
def get_job_data(job, average):
    """Retrieve data from a job that has already run.
    Args:
        job (Job): The job whose data you want.
        average (bool): If True, gets the data assuming data is an average.
                        If False, gets the data assuming it is for single shots.
    Return:
        list: List containing job result data.
    """
    job_results = job.result()  # timeout parameter set to 120 s
    result_data = []
    for i in range(len(job_results.results)):
        if average:  # get avg data
            result_data.append(np.real(job_results.get_memory(i)[qubit] * scale_factor))
        else:  # get single data
            result_data.append(job_results.get_memory(i)[:, qubit] * scale_factor)
    return result_data

def fit_function(x_values, y_values, function, init_params):
    fitparams, conv = curve_fit(function, x_values, y_values, init_params)
    y_fit = function(x_values, *fitparams)
    
    return fitparams, y_fit

def reshape_complex_vec(vec):
    """Take in complex vector vec and return 2d array w/ real, imag entries. This is needed for the learning.
    Args:
        vec (list): complex vector of data
    Returns:
        list: vector w/ entries given by (real(vec], imag(vec))
    """
    length = len(vec)
    vec_reshaped = np.zeros((length, 2))
    for i in range(len(vec)):
        vec_reshaped[i] = [np.real(vec[i]), np.imag(vec[i])]
    return vec_reshaped

def discriminate(data) :
    zero_data = data[0]
    one_data = data[1]
    two_data = data[2]
    zero_data_reshaped = reshape_complex_vec(zero_data)
    one_data_reshaped = reshape_complex_vec(one_data)
    two_data_reshaped = reshape_complex_vec(two_data)
    IQ_012_data = np.concatenate((zero_data_reshaped, one_data_reshaped, two_data_reshaped))
    state_012 = np.zeros(NUM_SHOTS)  
    state_012 = np.concatenate((state_012, np.ones(NUM_SHOTS)))
    state_012 = np.concatenate((state_012, 2 * np.ones(NUM_SHOTS)))
    IQ_012_train, IQ_012_test, state_012_train, state_012_test = train_test_split(IQ_012_data, state_012, test_size=0.3)
    LDA_012 = LinearDiscriminantAnalysis()
    LDA_012.fit(IQ_012_train, state_012_train)
    return LDA_012

def count(data, discriminator):
    sched_data = []
    for i in range(len(data)):
        sched_data.append(reshape_complex_vec(data[i]))
    discrim_data = []
    for j in range(len(sched_data)):
        discrim_data.append(discriminator.predict(sched_data[j]))
        #print('predicting', j, end='\r')
    final_result = []
    for k in range(len(discrim_data)):
        result = {'0': 0, '1': 0, '2': 0}
        for l in range(len(discrim_data[k])):
            if discrim_data[k][l] == 0.0:
                result['0'] += 1
            elif discrim_data[k][l] == 1.0:
                result['1'] += 1
            elif discrim_data[k][l] == 2.0:
                result['2'] += 1
            else:
                print('Unexpected behavior')
        final_result.append(result)
    return final_result

def IQ_012_plot(x_min, x_max, y_min, y_max, data):
    """Helper function for plotting IQ plane for 0, 1, 2. Limits of plot given
    as arguments."""
    # zero data plotted in blue
    zero_data = data[0]
    one_data = data[1]
    two_data = data[2]
    alpha = 1
    size = 10
    plt.scatter(np.real(zero_data), np.imag(zero_data),
                s=size, cmap='viridis', c='blue', alpha=alpha, label=r'$|0\rangle$')
    # one data plotted in red
    plt.scatter(np.real(one_data), np.imag(one_data),
                s=size, cmap='viridis', c='red', alpha=alpha, label=r'$|1\rangle$')
    # two data plotted in green
    plt.scatter(np.real(two_data), np.imag(two_data),
                s=size, cmap='viridis', c='green', alpha=alpha, label=r'$|2\rangle$')

    # Plot a large dot for the average result of the 0, 1 and 2 states.
    mean_zero = np.mean(zero_data) # takes mean of both real and imaginary parts
    mean_one = np.mean(one_data)
    mean_two = np.mean(two_data)
    mean_alpha = 1
    mean_size = 100
    plt.scatter(np.real(mean_zero), np.imag(mean_zero),
                s=mean_size, cmap='viridis', c='black',alpha=mean_alpha)
    plt.scatter(np.real(mean_one), np.imag(mean_one),
                s=mean_size, cmap='viridis', c='black',alpha=mean_alpha)
    plt.scatter(np.real(mean_two), np.imag(mean_two),
                s=mean_size, cmap='viridis', c='black',alpha=mean_alpha)

    plt.xlim(x_min, x_max)
    plt.ylim(y_min,y_max)
    plt.legend(fontsize=15)
    plt.grid()
    plt.ylabel('I [a.u.]', fontsize=15)
    plt.xlabel('Q [a.u.]', fontsize=15)
    plt.title("0-1-2 discrimination", fontsize=15)

#mitigate
from cvxopt import matrix, solvers
def mitigated_population(p, C):
    P = matrix(np.transpose(C).dot(C))
    q = matrix(-np.transpose(p).dot(C))
    G = matrix([[-1.0,0.0,0.0],[0.0,-1.0,0.0],[0.0,0.0,-1.0]])
    h = matrix([0.0, 0.0, 0.0])
    A = matrix([1.0, 1.0, 1.0], (1,3))
    b = matrix(1.0)
    solvers.options['show_progress'] = False
    sol=solvers.qp(P, q, G, h, A, b)
    # return np.asarray(sol['x'])
    return np.array([e for e in sol['x']])

def get_population(data, discriminator):
    drag_values = count(data, discriminator)
    return np.asarray([[val['0']/NUM_SHOTS, val['1']/NUM_SHOTS, val['2']/NUM_SHOTS] for val in drag_values])

def get_mitigated_population(data, discriminator, confusion_matrix):
    population = get_population(data, discriminator)
    mitigated = [mitigated_population(pop, confusion_matrix) for pop in population]
    return np.asarray(mitigated)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

#loss function
def loss_func(param, theta, phi, p_exp, N_data = 97):
  p_model = [population_model(theta[i], phi[i], *param, order) for i in range(N_data)]
  return mean_squared_error(p_model, population, squared = 'False')

In [5]:
#define matrix representaion of some gates
def Z01(phi):
    return np.array([[np.exp(-1j * phi), 0, 0],
                     [0, 1, 0],
                     [0, 0, 1]])

def Z12(phi):
    return np.array([[1, 0, 0],
                     [0, 1, 0],
                     [0, 0, np.exp(1j * phi)]])

def X01(theta):
    return np.array([[np.cos(theta / 2), -1j * np.sin(theta / 2), 0],
                     [-1j * np.sin(theta / 2), np.cos(theta / 2), 0],
                     [0, 0, 1]])


def X12(theta):
    return np.array([[1, 0, 0],
                     [0, np.cos(theta / 2), -1j * np.sin(theta / 2)],
                     [0, -1j * np.sin(theta / 2), np.cos(theta / 2)]])

def R01(theta, phi, phi2):
  R01_theory = Z01(phi) @ X01(theta) @ Z01(-phi)
  return Z12(phi2) @ R01_theory

def R12(theta, phi, phi0, phi2):
  R12_theory = Z12(phi) @ X12(theta) @ Z12(-phi)
  return Z01(phi0) @ R12_theory @ Z12(phi2)

def U_d(phi_1, phi_2, phi_3):
    return np.array([[np.exp(1j*phi_1), 0, 0],
                   [0, np.exp(1j*phi_2), 0],
                   [0, 0, np.exp(1j*phi_3)]])

def R01_theory(theta, phi):
    return Z01(phi) @ X01(theta) @ Z01(-phi)

def R12_theory(theta, phi):
    return Z12(phi) @ X12(theta) @ Z12(-phi)
def population_model(thetas, phis, phi2_01, phi0_12, phi2_12, order):
    ground = np.array([[1], [0], [0]])
    psi =  R12(np.pi/2, 0, phi0_12, phi2_12) @ R01(np.pi/2, 0, phi2_01) @ ground
    i = 0
    for level in order:
        if level == '1':
            psi =  R01(thetas[i], phis[i], phi2_01) @ psi
        else :
            psi = R12(thetas[i], phis[i], phi0_12, phi2_12) @ psi
        i = i + 1
    return [np.abs(psi[0][0])**2, np.abs(psi[1][0])**2, np.abs(psi[2][0])**2]

In [6]:
amp01 = 0.38639985577931224
#some useful pulse
with pulse.build(backend=backend, default_alignment='sequential', name=r'$X_{\pi}^{(12)}$') as xpi12:
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(f12, drive_chan)
    pulse.play(pulse.Gaussian(duration=dur12,
                            amp=amp12,
                            sigma=sig12,
                            name=r'$X_{\pi}^{(12)}$'), drive_chan)

with pulse.build(backend=backend, default_alignment='sequential', name=r'$X_{\pi/2}^{(12)}$') as x90_12:
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(f12, drive_chan)
    pulse.play(pulse.Gaussian(duration=dur12,
                            amp=amp12/2,
                            sigma=sig12,
                            name=r'$X_{\pi/2}^{(12)}$'), drive_chan)
    
with pulse.build(backend=backend, default_alignment='sequential', name=r'$X_{\pi}^{(01)}$') as xpi01:
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(f01, drive_chan)
    pulse.play(pulse.Drag(duration=dur01,
                               amp=amp01,
                               sigma=sig01,
                               beta=-1.0509653466942652,
                               angle=0.0,
                               name=r'$X_{\pi}^{(01)}$'), drive_chan)

with pulse.build(backend=backend, default_alignment='sequential', name=r'$X_{\pi/2}^{(01)}$') as x90_01:
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(f01, drive_chan)
    pulse.play(pulse.Drag(duration=dur01,
                               amp=0.19028167169061994,
                               sigma=sig01,
                               beta=-1.0849508117205746,
                               angle=0.022530958997040784,
                               name=r'$X_{\pi/2}^{(01)}$'), drive_chan)   

#define a pulse rotating theta around n-axis in subspace 0-1 and 1-2

def r01(theta, phi):
    with pulse.build(backend=backend, default_alignment='sequential', name=r'$R^{(01)}$') as r01_pulse:
        drive_chan = pulse.drive_channel(qubit)
        pulse.set_frequency(f01, drive_chan)
        with pulse.phase_offset(-phi, drive_chan):
            pulse.play(pulse.Gaussian(duration=dur01,
                                      amp=amp01*theta/np.pi,
                                      sigma=sig01,
                                      name=r'$X_{\pi/2}^{(01)}$'), drive_chan)

    return r01_pulse

def r12(theta, phi):
    with pulse.build(backend=backend, default_alignment='sequential',
                     name=r'$R^{(12)}$') as r12_pulse:
        drive_chan = pulse.drive_channel(qubit)
        pulse.set_frequency(f12, drive_chan)
        with pulse.phase_offset(-phi, drive_chan):
            pulse.play(pulse.Gaussian(duration=dur12,
                                      amp=amp12*theta/np.pi,
                                      sigma=sig12,
                                      name=r'$X_{\pi/2}^{(12)}$'), drive_chan)

    return r12_pulse

# Create the three circuits
Xpi01 = Gate("$X_{\pi}^{01}$", 1, [])
Xpi12 = Gate("$X_{\pi}^{12}$", 1, [])
X90_01 = Gate(r'$X_{\pi/2}^{(01)}$', 1, [])
X90_12 = Gate(r'$X_{\pi/2}^{(12)}$', 1, [])
# 0 state
zero = QuantumCircuit(1, 1)
zero.measure(0, 0)

# 1 state
one = QuantumCircuit(1, 1)

#one.append(Xpi01, [0])
#one.add_calibration(Xpi01, (0,), r01(pi, 0), [])
one.x(0)
one.measure(0, 0)

# 2 state 
two = QuantumCircuit(1, 1)
two.x(0)
#two.append(Xpi01, [0])
two.append(Xpi12, [0])
two.measure(0, 0)
#two.add_calibration(Xpi01, (0,), r01(pi, 0), [])
two.add_calibration(Xpi12, (0,), r12(pi, 0), [])

discr_circ = [zero, one, two]

In [7]:
def reconstruct_clifford_index(index):
    theta_1, theta_2, theta_3, phi_1, phi_2, phi_3, phi_4, phi_5, phi_6 = clifford_params[index]
    return U_d(phi_6, phi_5, phi_4) @ R01_theory(theta_3, phi_3) @ R12_theory(theta_2, phi_2) @ R01_theory(theta_1, phi_1)

def population_theory(index):
    ground = np.array([[1], [0], [0]])
    psi = Clifford3[index] @ ground
    return np.array([np.abs(psi[0][0])**2, np.abs(psi[1][0])**2, np.abs(psi[2][0])**2])

def reconstruct_clifford_parameter(params):
    theta_1, theta_2, theta_3, phi_1, phi_2, phi_3, phi_4, phi_5, phi_6 = params
    return U_d(phi_6, phi_5, phi_4) @ R01_theory(theta_3, phi_3) @ R12_theory(theta_2, phi_2) @ R01_theory(theta_1, phi_1)

In [8]:
exp = np.exp
d = 3 # d-dimensional Lie group, here Clifford C_3 d=3
omega = exp(2*pi*1j/d) # root of unity

sqrt = np.sqrt
exp = np.exp
pi = np.pi
arr = np.array

Hdm = 1/sqrt(3)*arr([
    [1, 1       ,   1        ],
    [1, omega   ,   omega**2 ],
    [1, omega**2,   omega    ]
])

Sdg = arr([
    [1,     0,    0        ],
    [0,     1,    0        ],
    [0,     0,    omega    ]
])


def sum_index(i):
    return 2**(i+1)-2

A = []
for i in range(12):
    if i == 0:
        A.append("H")
        A.append("S")
    else: 
        for j in range(sum_index(i-1),sum_index(i)):
            A.append(A[j]+"H")
            A.append(A[j]+"S")
            
A_matrix = []
for i in range(len(A)):
    result = np.complex128(np.zeros([3,3]))
    for j in range(len(A[i]),):
        if j == 0:
            if A[i][j] == 'H': result += Hdm
            elif A[i][j] == 'S': result += Sdg
        else:
            if A[i][j] == 'H': result = result@Hdm
            elif A[i][j] =='S': result = result@Sdg
    A_matrix.append(result)
    
A_matrix_flitered = []
A_matrix_flitered_index = []
for i in range(len(A_matrix)):
    if i == 0:
        A_matrix_flitered.append(A_matrix[i])
        A_matrix_flitered_index.append(i)
    else:
        flag = True
        for j in range(len(A_matrix_flitered)):
            if (np.round(A_matrix[i] - A_matrix_flitered[j], decimals = 10) == 0).all():
                flag = False
                break
            else: 
                continue
        if flag == True:
            A_matrix_flitered.append(A_matrix[i])
            A_matrix_flitered_index.append(i)
            
for i in range(len(A_matrix_flitered)):
    A_matrix_flitered[i] = np.round(A_matrix_flitered[i], decimals = 10)
    
Clifford3 = A_matrix_flitered
Clifford3_latex = [array_to_latex(mat) for mat in Clifford3]

In [9]:
len(Clifford3)

937

In [10]:
def is_global_phase(mat_1, mat_2):
    for i in range(3):
        if mat_1[0][i] != 0:
            arg_1 = np.angle(mat_1[0][i])
            arg_2 = np.angle(mat_2[0][i])
            break
        else:
            continue
    identity = np.identity(3, np.complex128)
    zeros = np.zeros([3,3], np.complex128)
    diff = mat_1 * np.exp(1j*arg_2) - mat_2 * np.exp(1j*arg_1)
    if (np.round(diff, decimals=10)==0).all():
        return True
    else:
        return False
        

In [11]:
global_phase = []
for i in range(len(Clifford3)):
    wrong = [i]
    for j in range(i+1, len(Clifford3)):
        if is_global_phase(Clifford3[i], Clifford3[j]):
            wrong.append(j)
    global_phase.append(wrong)



In [12]:
no_need = set()
for i in range(len(global_phase)):
    for j in range(1, len(global_phase[i])):
        no_need.add(global_phase[i][j])
        
len(no_need)

721

In [13]:
index = np.arange(0, len(Clifford3))

right_index = np.delete(index, list(no_need))
print(len(right_index))

Clifford3_update = []
for i in right_index:
    Clifford3_update.append(Clifford3[i])

216


In [14]:
pairs_inv = []
for i in range(len(Clifford3_update)):
    res = [i]
    for j in range(0, len(Clifford3_update)):
        mat = Clifford3_update[i]@Clifford3_update[j]
        iden = np.identity(3, np.complex128)
        arg = np.angle(mat[0][0])
        #print(arg)
        iden_2 = iden * exp(1j*arg)
        if (np.round(mat - iden_2, decimals=10) == 0).all():
            #print(i)
            #print(arg)
            res.append(j)
            #res.append(arg, decimals=9)
    pairs_inv.append(res)

In [15]:
def check_pairs(lst):
    if lst[1] >= lst[0]:
        return True
    else:
        return False
short_pairs_inv = []
for pair in pairs_inv:
    if check_pairs(pair):
        #print(pair)
        short_pairs_inv.append(pair)

In [16]:
len(short_pairs_inv)

113

In [17]:
Clifford3_final = Clifford3_update

In [18]:
def multi(lst):
    mat = np.identity(3, np.complex128)
    for i in lst:
        mat = mat @ Clifford3_final[i]
    return mat

In [19]:
def is_global_phase(mat_1, mat_2):
    for i in range(3):
        if mat_1[0][i] != 0:
            arg_1 = np.angle(mat_1[0][i])
            arg_2 = np.angle(mat_2[0][i])
            break
        else:
            continue
    #print(arg_1, arg_2)
    identity = np.identity(3, np.complex128)
    zeros = np.zeros([3,3], np.complex128)
    diff = mat_1 * np.exp(1j*arg_2) - mat_2 * np.exp(1j*arg_1)
    if (np.round(diff, decimals=6)==0).all():
        return True
    else:
        return False
        

In [20]:
def find_index(mat):
    index = -1
    for i in range(216):
        if is_global_phase(mat, Clifford3_final[i])==True:
            index = i
    return index

def find_inv_index(index):
    inv_index = -1
    for pair in pairs_inv:
        if pair[0] == index:
            inv_index = pair[1]
        if pair[1] == index:
            inv_index = pair[0]
    return inv_index

def rmb(length):
    sequence = np.random.randint(0, 216, length)
    mat = multi(np.flip(sequence))
    index_mat = find_index(mat)
    inv_index = find_inv_index(index_mat)
    while inv_index == -1:
        sequence = np.random.randint(0, 216, length)
        mat = multi(np.flip(sequence))
        index_mat = find_index(mat)
        inv_index = find_inv_index(index_mat)
    rmb_sequence = np.concatenate((sequence, np.array([inv_index])))
    return rmb_sequence

def check_is_inv(lst):
    rand_index = lst[:len(lst)-1]
    mat = multi(np.flip(rand_index))
    ground = np.array([[1], [0], [0]])
    psi = Clifford3_final[lst[-1]]@mat@ground
    #print(psi)
    if np.round(np.abs(psi[0])) == 1:
        return True
    else:
        return False
    
def psi(lst):
    rand_index = lst[:len(lst)-1]
    mat = multi(np.flip(rand_index))
    ground = np.array([[1], [0], [0]])
    psi = Clifford3_final[lst[-1]]@mat@ground
    return array_to_latex(psi)

In [21]:
def interleaved_rmb(index, length):
    rmb_seq = rmb(length)
    #print(rmb_seq)
    for i in range(1, length + 1, 2):
        rmb_seq_update = np.insert(rmb_seq, [i for i in range(1, length+1)], index)
    rmb_seq_update_new = np.delete(rmb_seq_update, rmb_seq_update.shape[0]-1)
    #print(rmb_seq_update)
    #print(rmb_seq_update_new)
    mat = multi(np.flip(rmb_seq_update_new))
    index_mat = find_index(mat)
    inv_index = find_inv_index(index_mat)
    while inv_index == -1:
        rmb_seq = rmb(length)
        for i in range(1, length + 1, 2):
            rmb_seq_update = np.insert(rmb_seq, [i for i in range(1, length+1)], index)
        rmb_seq_update_new = np.delete(rmb_seq_update, rmb_seq_update.shape[0]-1)
        mat = multi(np.flip(rmb_seq_update_new))
        index_mat = find_index(mat)
        inv_index = find_inv_index(index_mat)
    interleaved_rmb_sequence = np.concatenate((rmb_seq_update_new, np.array([inv_index])))
    return interleaved_rmb_sequence

In [22]:
def get_parameter(index):
    U = Clifford3_final[index]
    if np.round(abs(np.absolute(U[2, 2])), 6) == 1:
        if np.round(abs(np.absolute(U[0, 0])), 6) != 0:
            theta_1 = phi_1 = theta_2 = phi_2 = 0
            phi_4 = np.angle(U[2, 2])
            phi_5 = np.angle(U[1, 1])
            phi_6 = np.angle(U[0, 0])
            # phi_3 = phi_6 - pi/2 - np.angle(U[0, 1])
            phi_3 = np.angle(U[1, 0]) - phi_5 + pi/2
            theta_3 = 2*np.arccos(np.round(np.absolute(U[1, 1]), 6))
        else:
            theta_1 = phi_1 = theta_2 = phi_2 = phi_3 = 0
            theta_3 = 2*np.arccos(np.round(np.absolute(U[1, 1]), 6))
            phi_4 = np.angle(U[2, 2])
            phi_6 = np.angle(U[0, 1]) + phi_3 + pi/2
            phi_5 = np.angle(U[1, 0]) - phi_3 + pi/2
    elif np.round(abs(np.absolute(U[2, 2])), 6) == 0:
        theta_1 = 2*np.arccos(np.round(np.absolute(U[2, 1]), 6))
        theta_2 = pi
        theta_3 = 2*np.arccos(np.round(np.absolute(U[1, 2]), 6))
        phi_1 = phi_2 = phi_3 = 0
        if np.round(abs(np.absolute(U[2, 0])), 6) != 0:
            phi_4 = np.angle(-U[2, 0])
            if np.round(abs(np.absolute(U[0, 2])), 6) != 0:
                phi_5 = np.angle(-U[1, 1])
                phi_6 = np.angle(-U[0, 2])
            else:
                phi_5 = np.angle(U[1, 2]) + pi/2
                phi_6 = np.angle(U[0, 1]) + pi/2 
        if np.round(abs(np.absolute(U[1, 0])), 6) != 0:
            phi_4 = np.angle(U[2, 1]) + pi/2
            phi_5 = np.angle(U[1, 0]) + pi/2
            phi_6 = np.angle(-U[0, 2])
        if np.round(abs(np.absolute(U[0, 0])), 6) != 0:
            phi_4 = np.angle(U[2, 1]) + pi/2
            phi_5 = np.angle(U[1, 2]) + pi/2
            phi_6 = np.angle(U[1, 1])
        
    else:
        phi_4 = np.angle(U[2, 2])
        theta_2 = 2*np.arccos(np.round(np.absolute(U[2, 2]), 6))
        phi_2 = np.angle(U[2, 1]) - phi_4 + pi/2
        phi_1 = np.angle(-U[2, 0]) - phi_2 - phi_4
        theta_1 = 2*np.arccos(np.round(np.absolute(U[2, 1])/np.sin(theta_2/2), 6))
        theta_3 = 2*np.arccos(np.round(np.absolute(U[1, 2])/np.sin(theta_2/2), 6))
        phi_5 = np.angle(U[1, 2]) + phi_2 + pi/2
        phi_3 = np.angle(np.cos(theta_1/2) * np.cos(theta_2/2) * np.cos(theta_3/2) - U[1, 1]*np.exp(-1j*phi_5)) + phi_1
        phi_6 = np.angle(-U[0, 2]) + phi_3 + phi_2
    return theta_1, theta_2, theta_3, phi_1, phi_2, phi_3, phi_4, phi_5, phi_6

def reconstruct(theta_1, theta_2, theta_3, phi_1, phi_2, phi_3, phi_4, phi_5, phi_6):
    return U_d(phi_6, phi_5, phi_4) @ R01_theory(theta_3, phi_3) @ R12_theory(theta_2, phi_2) @ R01_theory(theta_1, phi_1)

In [23]:
def find_bug():
    bug = []
    for i in range(216):
        mat_gen = Clifford3_final[i]
        theta_1, theta_2, theta_3, phi_1, phi_2, phi_3, phi_4, phi_5, phi_6 = get_parameter(i)
        mat_reconstruct = reconstruct(theta_1, theta_2, theta_3, phi_1, phi_2, phi_3, phi_4, phi_5, phi_6)
        diff = np.round(mat_gen-mat_reconstruct, 5)
        if np.absolute(np.sum(diff)) != 0:
            bug.append(i)
    return bug

bug = find_bug()

bug

[]

In [24]:
#data = [get_parameter(i) for i in range(216)]

In [25]:
#np.savetxt('clifford_parameter_update.csv', data, delimiter=',')

In [26]:
clifford3_params = np.genfromtxt('clifford_parameter_update.csv', delimiter=',')

In [27]:
def create_pulse_sequence(params_sequence):
    phase_01, phase_12 = 0, 0
    #print(alpha, beta, gamma)
    with pulse.build(backend=backend, default_alignment='sequential') as pulse_sequence:
        drive_channel = pulse.drive_channel(qubit)
        pulse.delay(0, drive_channel)
    for params in params_sequence:
        #print(params)
        theta_1, theta_2, theta_3, phi_1, phi_2, phi_3, phi_4, phi_5, phi_6 = params
        
        with pulse.build(backend=backend, default_alignment='sequential') as sched:
            drive_channel = pulse.drive_channel(qubit)
            
            #R01(theta, phi)
            pulse.set_frequency(f01, drive_channel)
            pulse.call(r01(theta_1, phi_1 + phase_01))
            phase_12 = phase_12 + alpha
            
            #R12(theta, phi)           
            pulse.set_frequency(f12, drive_channel)
            pulse.call(r12(theta_2, phi_2 + phase_12))
            phase_01 = phase_01 + beta
            phase_12 = phase_12 + gamma
            
            #R01(theta, phi)      
            pulse.set_frequency(f01, drive_channel)
            pulse.call(r01(theta_3, phi_3 + phase_01))
            phase_12 = phase_12 + alpha
            
            #U_d(phi6, phi5, phi4)
            phase_01 = phase_01 + phi_6 - phi_5
            phase_12 = phase_12 + phi_5 - phi_4

        pulse_sequence = pulse_sequence + sched
        #print(phase_01, phase_12)
    return pulse_sequence

In [28]:
def rmb_circ(length):
    rmb_seq = 0
    if length == 0:
        qc = QuantumCircuit(1,1)
        qc.measure(0,0)
    else:
        rmb_seq = rmb(length)
        rmb_params = [clifford3_params[index] for index in rmb_seq]
        #print(rmb_seq)
        rmb_pulse = create_pulse_sequence(rmb_params)
        rmb_gate = Gate('RMB', 1, [])
        qc = QuantumCircuit(1,1)
        qc.append(rmb_gate, [0])
        qc.add_calibration(rmb_gate, (0,), rmb_pulse, [])
        #qc.measure(0, 0)
    return qc, rmb_seq

In [29]:
def interleaved_rmb_circ(index, length):
    interleaved_rmb_seq = 0
    if length == 0:
        qc = QuantumCircuit(1,1)
        qc.measure(0,0)
    else:
        interleaved_rmb_seq = interleaved_rmb(index, length)
        #print(interleaved_rmb_seq)
        interleaved_rmb_params = [clifford3_params[index] for index in interleaved_rmb_seq]
        interleaved_rmb_pulse = create_pulse_sequence(interleaved_rmb_params)
        interleaved_rmb_gate = Gate('IRMB', 1, [])
        qc = QuantumCircuit(1,1)
        qc.append(interleaved_rmb_gate, [0])
        qc.add_calibration(interleaved_rmb_gate, (0,), interleaved_rmb_pulse, [])
        qc.measure(0, 0)
    return qc

In [30]:
alpha, beta, gamma = np.array([0.61004155, 0.72967769, 6.24307464])

In [35]:
array_to_latex(Clifford3_final[116])
array_to_latex(Clifford3_final[116])

<IPython.core.display.Latex object>

In [35]:
#average of 50 jobs, each job runs from length 1 to 97
index = 0
list_id = []
for i in range(50):
    interleaved_rmb_circs = [interleaved_rmb_circ(index, length) for length in range(1, 98)]
    interleaved_rmb_circs = discr_circ + interleaved_rmb_circs
    interleaved_rmb_job = backend.run(interleaved_rmb_circs, meas_level=1, meas_return='single', shots=NUM_SHOTS)
    print(interleaved_rmb_job.job_id())
    list_id.append(interleaved_rmb_job.job_id())

cjktekvijvusg3r8u5fg
cjktg0fijvusg3r8umr0
cjkth9kvcjlre5cc28rg
cjktilcvcjlre5cc2ua0
cjktk2ke6pin8ppkdfv0
cjktlb4e6pin8ppke3c0
cjktml76hmsgmh1r7350
cjktntcvcjlre5cc5cj0
cjktp64vcjlre5cc5tg0
cjktqckvcjlre5cc6er0
cjktrivijvusg3r943i0
cjktsov6hmsgmh1r9scg
cjktu9n6hmsgmh1rags0
cjktvpv6hmsgmh1rb750
cjku15ce6pin8ppkjklg
cjku29c4jk0qbksgbs5g
cjku3en6hmsgmh1rd3og
cjku4lkvcjlre5ccbgpg
cjku5t76hmsgmh1reebg
cjku71nijvusg3r99skg
cjku89k4jk0qbksgf0sg
cjku9f4e6pin8ppknvk0
cjkual4e6pin8ppkol4g
cjkuboov6od45s3tjdng
cjkuctnijvusg3r9d20g
cjkue1f6hmsgmh1rilj0
cjkuf644jk0qbksginig
cjkugck4jk0qbksgjas0
cjkuhin6hmsgmh1rkhig
cjkuipk4jk0qbksgkis0
cjkujv0v6od45s3to6c0
cjkul3v6hmsgmh1rmedg
cjkum90v6od45s3tpe60
cjkundkvcjlre5cclmjg
cjkuoifijvusg3r9jatg
cjkupo4e6pin8ppl0rr0
cjkuqufijvusg3r9kgv0
cjkus38v6od45s3tsjug
cjkut9vijvusg3r9lpb0
cjkuuecvcjlre5ccpb70
cjkuvin6hmsgmh1rrtpg
cjkv0on6hmsgmh1rsj9g
cjkv1svijvusg3r9oa3g
cjkv394vcjlre5ccs720
cjkv4jfijvusg3r9q0kg
cjkv5qsvcjlre5cctlu0
cjkv6vkvcjlre5ccu8n0
cjkv84n6hmsgm

In [33]:
populations = []
mats = []
for id in list_id:
    interleaved_rmb_job = provider.backend.retrieve_job(id)
    interleaved_rmb_data = get_job_data(interleaved_rmb_job, average=False)
    discrim_data = interleaved_rmb_data[:3]
    exp_data = interleaved_rmb_data[3:]
    LDA_012 = discriminate(discrim_data)
    discr_data = count(discrim_data, LDA_012)
    conf_mat = [[discr_data[i]['0'] / NUM_SHOTS, discr_data[i]['1'] / NUM_SHOTS, discr_data[i]['2'] / NUM_SHOTS] for i in
                   range(np.shape(discr_data)[0])] 
    mats.append(conf_mat)
    population = get_mitigated_population(exp_data, LDA_012, conf_mat)
    populations.append(population)

In [34]:
for mat in mats:
    if min(mat[0][0], mat[1][1], mat[2][2]) < 0.85:
        print(mat)

In [35]:
rmb_populations = [populations[i][:, 0] for i in range(len(populations))]
rmb_populations = np.array([list(population) for population in rmb_populations])

In [36]:
average_populations = np.zeros((1, 97))
for i in range(len(populations)):
    average_populations = average_populations + populations[i][:, 0]


In [37]:
average_populations = average_populations/len(populations)
average_populations

array([[0.94017629, 0.95831169, 0.93055185, 0.90734985, 0.8733105 ,
        0.85423299, 0.84861584, 0.84008671, 0.81782451, 0.79661847,
        0.80824819, 0.7951763 , 0.77688409, 0.76200678, 0.72177085,
        0.75261762, 0.6978516 , 0.70754494, 0.71675304, 0.69049258,
        0.66035096, 0.68235017, 0.64663907, 0.67309299, 0.65892403,
        0.62161483, 0.62438712, 0.63675119, 0.61473538, 0.59503182,
        0.61227516, 0.59908328, 0.59024982, 0.59457959, 0.58326441,
        0.57459931, 0.55806841, 0.5541779 , 0.56507128, 0.54722177,
        0.55770686, 0.54394382, 0.55722944, 0.56883047, 0.52170278,
        0.54387074, 0.5397453 , 0.52441954, 0.53514929, 0.51582347,
        0.51003392, 0.49763417, 0.51426252, 0.53656255, 0.49267622,
        0.52241166, 0.51528622, 0.51245382, 0.49395894, 0.504454  ,
        0.5019167 , 0.52568112, 0.49923196, 0.4590564 , 0.51114028,
        0.49932163, 0.49306233, 0.47828481, 0.47085513, 0.48797047,
        0.48080389, 0.451844  , 0.46543303, 0.47